**Imports**

PyTorch Geometric

In [1]:
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric 

Looking in links: https://pytorch-geometric.com/whl/torch-1.11.0+cu113.html
     |████████████████████████████████| 7.9 MB 4.6 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.11.0+cu113.html
     |████████████████████████████████| 3.5 MB 5.0 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.11.0+cu113.html
     |████████████████████████████████| 2.5 MB 5.1 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.11.0+cu113.html
     |████████████████████████████████| 750 kB 4.5 MB/s 
     |████████████████████████████████| 407 kB 5.3 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.4-py3-none-any.whl size=616603 sha256=2c9b493bb29732bd5f6d34d0c22aa03ea7e9979c6050f813db8007750cbd41be
  Stored in directory: /root/.cache/pip/wheels/18/a6/a4/ca18c3051fcead866fe7b85700ee2240d883562a1bc70ce421
Successfully built torch-geometric


In [2]:
import pandas as pd
import numpy as np
import torch.nn.functional as F 

from torch_geometric.data import InMemoryDataset, download_url, Data
from datetime import datetime
from torch.nn import Linear
from IPython.display import Javascript
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool

**Mount GDrive**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ls

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
action_types.csv              participants_activity_anomalies.csv
activities.csv                patients.csv
activity_types.csv            scores.csv
anomalies.csv                 scores_predictions.csv
anomaly_types.csv             sensor_locations.csv
diagnosis_predictions.csv     sensors.csv
diagnosis_types.csv           tasks.csv
events.csv                    task_types.csv
overt_error_action_types.csv  trajectory_anomalies.csv


In [16]:
pd.read_csv('drive/MyDrive/CASAS400/activities.csv')

,patient,activity_type,start,end
0,88,1,09:21:17.2387,09:24:10.78761
1,88,2,09:25:15.5674,09:28:42.21513
2,88,3,09:30:21.23501,09:33:50.33897
3,88,4,09:34:53.77164,09:42:28.82678
4,88,5,09:43:36.54254,09:44:33.83539
...,...,...,...,...
5274,329,11,14:36:47.11877,14:40:02.51782
5275,329,12,14:44:22.30159,14:47:07.77214
5276,329,13,14:48:19.93743,14:52:49.75592
5277,329,14,14:53:32.62822,15:02:49.75547


# Riconoscimento di diagnosi paziente

In [ ]:
def task_activity_sensors(tasks, events, sensors, activities, patient):

  # Seleziona i task-activity eseguiti dal paziente 'patient'
  task_activity = tasks.query('patient == ' + patient)

  # Seleziona il tempo di inizio e fine delle attività svolte dal paziente 'patient'
  activities_patient = activities.query('patient == ' + patient)
  start_time = activities_patient.iloc[0]['start']
  end_time = activities_patient.iloc[len(activities_patient) - 1]['end']

  # Query nel dataframe events per selezionare solo i sensori utilizzati nelle attività
  sensors_events = events.query('patient == ' + patient + ' and time >= "' + start_time + '" and time <= "' + end_time + '"')

  # Matrice risultato che contiene i dati dei nodi
  res_node = pd.DataFrame(columns=['features'])
  res_edge = pd.DataFrame(columns=['times']) 

  # Dimensione dell'array delle features
  dim_features = len(sensors)

  # Dizionario che contiene la posizione del sensore nell'array delle features
  dict_sens = {}

  for i in range(0, dim_features):
    dict_sens[sensors.iloc[i]['sensor_id']] = i

  # Indice del task-activity
  i = 0

  # Numero edge del grafo
  num_edge = 0

  # Nome del task-activity precedente
  pair_str_prev = ""

  # Flag per salvataggio features di nodo e arco
  ins_upd = False
  last_node = False

  for i in range(0, len(task_activity)):

    # Controlla se il task-activity attuale è l'ultimo
    if ((i + 1) >= len(task_activity)):
      last_node = True

    # Nome del task-activity attuale
    pair_str = str(task_activity.iloc[i]['task']) + '-' + str(task_activity.iloc[i]['activity'])

    # Controlla se il task-activity attuale e quello precedente sono diversi
    if pair_str != pair_str_prev:

      # Viene salvato lo start time
      s_time = task_activity.iloc[i]['time']

      # Controlla se il task-activity attuale non è l'ultimo
      if not last_node:

        # Controlla se il task-activity attuale e quello successivo sono diversi
        if pair_str != str(task_activity.iloc[i + 1]['task']) + '-' + str(task_activity.iloc[i + 1]['activity']):

          # Viene salvato l'end time
          e_time = task_activity.iloc[i + 1]['time']

          ins_upd = True
          pair_str_prev = pair_str

        else:
          ins_upd = False
          pair_str_prev = pair_str
      
      else:

        # Viene salvato l'end time
        e_time = end_time

        ins_upd = True
        pair_str_prev = pair_str
    
    else:
      # Controlla se il task-activity attuale non è l'ultimo
      if not last_node:

        # Controlla se il task-activity attuale e quello successivo sono diversi
        if pair_str != str(task_activity.iloc[i + 1]['task']) + '-' + str(task_activity.iloc[i + 1]['activity']):

          # Viene salvato l'end time
          e_time = task_activity.iloc[i + 1]['time']

          ins_upd = True
          pair_str_prev = pair_str
        
        else:
          ins_upd = False
          pair_str_prev = pair_str
      
      else:

        # Viene salvato l'end time
        e_time = end_time

        ins_upd = True
        pair_str_prev = pair_str

    # Controlla se start ed end time sono stati trovati
    if ins_upd:

      # Controlla se il task-activity attuale non è l'ultimo
      if not last_node:

        # Aggiorna il numero di edge del grafo
        num_edge = num_edge + 1

        # Controlla se esiste un arco per il task-activity attuale 
        if pair_str in res_edge.index:

          # Aggiunge start ed end time all'arco
          res_edge.loc[pair_str, 'times'] += [((s_time, e_time), str(task_activity.iloc[i + 1]['task']) + '-' + str(task_activity.iloc[i + 1]['activity']))]
        
        else:
          # Crea un nuovo arco
          res_edge.loc[pair_str] = [[((s_time, e_time), str(task_activity.iloc[i + 1]['task']) + '-' + str(task_activity.iloc[i + 1]['activity']))]]

      # Controlla se non esiste un nodo per il task-activity attuale
      if not pair_str in res_node.index:

        # Crea un array vuoto per le fueatures del nodo
        features_node = np.zeros(dim_features, dtype=np.int8)

        # Seleziona i sensori utilizzati per il task-activity attuale
        task_activity_sensors = sensors_events.query('time >= "' + s_time + '" and time <= "' + e_time + '"')

        # Popola l'array delle features
        for _, s in task_activity_sensors.iterrows():
          features_node[dict_sens[s['sensor']]] = 1

        # Salva nel nodo l'array delle features
        res_node.loc[pair_str] = [features_node]
  
  return res_edge, res_node, num_edge

In [ ]:
sensors = pd.read_csv('drive/MyDrive/CASAS400/sensors.csv')
events = pd.read_csv('drive/MyDrive/CASAS400/events.csv')
tasks = pd.read_csv('drive/MyDrive/CASAS400/tasks.csv')
activities = pd.read_csv('drive/MyDrive/CASAS400/activities.csv')

patients = pd.read_csv('drive/MyDrive/CASAS400/activities.csv')['patient'].unique()

dataset = []

for i in range(0, len(patients)):

  # Crea i dataframe con le features degli edges e dei nodi
  edge, node, num_edge = task_activity_sensors(tasks, events, sensors, activities, str(patients[i]))

  # Salva la label associata al paziente
  label = pd.read_csv('drive/MyDrive/CASAS400/patients.csv').query('patient_id == ' + str(patients[i]))['diagnosis'].values

  if len(node) > 0:
    atr_node = np.zeros([len(node), len(node.iloc[0]['features'])])

    # Converte il dataframe dei nodi in una matrice
    for j in range(0, len(node)):
      atr_node[j] = node.iloc[j]['features']

    adj_matrix = np.empty([num_edge, 2])
    index_adj_matrix = 0

    # Crea la matrice di addiacenza dei nodi
    for j in range(0, len(edge)):
      for k in range(0, len(edge.iloc[j]['times'])):
        f_node = node.index.get_loc(edge.iloc[j].name)
        s_node = node.index.get_loc(edge.iloc[j]['times'][k][1])

        adj_matrix[index_adj_matrix] = np.array([f_node, s_node])
        index_adj_matrix = index_adj_matrix + 1
    
    # Salva la mtrice delle features e la matrice di addiacenza del grafo
    dataset.append(Data(x=torch.tensor(atr_node, dtype=torch.float),
                        edge_index=torch.tensor(adj_matrix, dtype=torch.long).t().contiguous(),
                        y=torch.tensor(label, dtype=torch.long)))

/usr/local/lib/python3.7/dist-packages/pandas/core/internals/blocks.py:937: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


In [ ]:
diagnosis = pd.read_csv('drive/MyDrive/CASAS400/diagnosis_types.csv')['diagnosis_id']

dim_label = pd.DataFrame(columns=['label', 'dimension'])

count = 0

for d in diagnosis:
  count = 0
  for item in dataset:
    if item.y[0] == d:
      count = count + 1
  
  dim_label = dim_label.append({'label': d, 'dimension': count}, ignore_index = True)

# Riconoscimento di attività paziente

In [24]:
def activity_graph(tasks, events, sensors, activities, patient, activity):

  # Seleziona il tempo di inizio e fine delle attività svolte dal paziente 'patient'
  activities_patient = activities.query('patient == ' + patient + ' and activity_type == ' + activity)
  start_time = activities_patient.iloc[0]['start']
  end_time = activities_patient.iloc[len(activities_patient) - 1]['end']

  # Query nel dataframe events per selezionare solo i sensori utilizzati nelle attività
  sensors_events = events.query('patient == ' + patient + ' and time >= "' + start_time + '" and time <= "' + end_time + '"')

  # Matrice risultato che contiene i dati dei nodi
  res_node = pd.DataFrame(columns=['features'])
  res_edge = pd.DataFrame(columns=['times']) 

  # Dimensione dell'array delle features
  dim_features = len(sensors)

  # Dizionario che contiene la posizione del sensore nell'array delle features
  dict_sens = {}

  for i in range(0, dim_features):
    dict_sens[sensors.iloc[i]['sensor_id']] = i

  task_activity = tasks.query('patient == ' + patient + ' and activity == ' + activity)

  # Indice del task-activity
  i = 0

  # Numero edge del grafo
  num_edge = 0

  # Nome del task-activity precedente
  pair_str_prev = ""

  # Flag per salvataggio features di nodo e arco
  ins_upd = False
  last_node = False

  for i in range(0, len(task_activity)):

    # Controlla se il task-activity attuale è l'ultimo
    if ((i + 1) >= len(task_activity)):
      last_node = True

    # Nome del task-activity attuale
    pair_str = str(task_activity.iloc[i]['task']) + '-' + str(task_activity.iloc[i]['activity'])

    # Controlla se il task-activity attuale e quello precedente sono diversi
    if pair_str != pair_str_prev:

      # Viene salvato lo start time
      s_time = task_activity.iloc[i]['time']

      # Controlla se il task-activity attuale non è l'ultimo
      if not last_node:

        # Controlla se il task-activity attuale e quello successivo sono diversi
        if pair_str != str(task_activity.iloc[i + 1]['task']) + '-' + str(task_activity.iloc[i + 1]['activity']):

          # Viene salvato l'end time
          e_time = task_activity.iloc[i + 1]['time']

          ins_upd = True
          pair_str_prev = pair_str

        else:
          ins_upd = False
          pair_str_prev = pair_str
      
      else:

        # Viene salvato l'end time
        e_time = end_time

        ins_upd = True
        pair_str_prev = pair_str
    
    else:
      # Controlla se il task-activity attuale non è l'ultimo
      if not last_node:

        # Controlla se il task-activity attuale e quello successivo sono diversi
        if pair_str != str(task_activity.iloc[i + 1]['task']) + '-' + str(task_activity.iloc[i + 1]['activity']):

          # Viene salvato l'end time
          e_time = task_activity.iloc[i + 1]['time']

          ins_upd = True
          pair_str_prev = pair_str
        
        else:
          ins_upd = False
          pair_str_prev = pair_str
      
      else:

        # Viene salvato l'end time
        e_time = end_time

        ins_upd = True
        pair_str_prev = pair_str

    # Controlla se start ed end time sono stati trovati
    if ins_upd:

      # Controlla se il task-activity attuale non è l'ultimo
      if not last_node:

        # Aggiorna il numero di edge del grafo
        num_edge = num_edge + 1

        # Controlla se esiste un arco per il task-activity attuale 
        if pair_str in res_edge.index:

          # Aggiunge start ed end time all'arco
          res_edge.loc[pair_str, 'times'] += [((s_time, e_time), str(task_activity.iloc[i + 1]['task']) + '-' + str(task_activity.iloc[i + 1]['activity']))]
        
        else:
          # Crea un nuovo arco
          res_edge.loc[pair_str] = [[((s_time, e_time), str(task_activity.iloc[i + 1]['task']) + '-' + str(task_activity.iloc[i + 1]['activity']))]]

      # Controlla se non esiste un nodo per il task-activity attuale
      if not pair_str in res_node.index:

        # Crea un array vuoto per le fueatures del nodo
        features_node = np.zeros(dim_features, dtype=np.int8)

        # Seleziona i sensori utilizzati per il task-activity attuale
        task_activity_sensors = sensors_events.query('time >= "' + s_time + '" and time <= "' + e_time + '"')

        # Popola l'array delle features
        for _, s in task_activity_sensors.iterrows():
          features_node[dict_sens[s['sensor']]] = 1

        # Salva nel nodo l'array delle features
        res_node.loc[pair_str] = [features_node]
  
  return res_edge, res_node, num_edge

In [26]:
sensors = pd.read_csv('drive/MyDrive/CASAS400/sensors.csv')
events = pd.read_csv('drive/MyDrive/CASAS400/events.csv')
tasks = pd.read_csv('drive/MyDrive/CASAS400/tasks.csv')
activities = pd.read_csv('drive/MyDrive/CASAS400/activities.csv')

patients = pd.read_csv('drive/MyDrive/CASAS400/activities.csv')['patient'].unique()

dataset = []

for i in range(0, len(patients)):

  patient_activities = activities.query('patient == ' + str(patients[i]))['activity_type'].unique()

  for act in patient_activities:

    # Salva la label associata al grafo dell'attività corrispondente
    label = np.array([act])

    # Crea i dataframe con le features degli edges e dei nodi
    edge, node, num_edge = activity_graph(tasks, events, sensors, activities, str(patients[i]), str(act))

    if len(node) > 0:
      atr_node = np.zeros([len(node), len(node.iloc[0]['features'])])

      # Converte il dataframe dei nodi in una matrice
      for k in range(0, len(node)):
        atr_node[k] = node.iloc[k]['features']

      adj_matrix = np.empty([num_edge, 2])
      index_adj_matrix = 0

      # Crea la matrice di addiacenza dei nodi
      for k in range(0, len(edge)):
        for x in range(0, len(edge.iloc[k]['times'])):
          f_node = node.index.get_loc(edge.iloc[k].name)
          s_node = node.index.get_loc(edge.iloc[k]['times'][x][1])

          adj_matrix[index_adj_matrix] = np.array([f_node, s_node])
          index_adj_matrix = index_adj_matrix + 1
      
      # Salva la mtrice delle features e la matrice di addiacenza del grafo
      dataset.append(Data(x=torch.tensor(atr_node, dtype=torch.float),
                          edge_index=torch.tensor(adj_matrix, dtype=torch.long).t().contiguous(),
                          y=torch.tensor(label, dtype=torch.long)))

1


/usr/local/lib/python3.7/dist-packages/pandas/core/internals/blocks.py:937: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


Output streaming troncato alle ultime 5000 righe.
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
1
2
3
4
5
7
6
8
9
10
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
12
13
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
5
8
10
11
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
14
15
16
1
2
3
4
5
6
7
8
9
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
11
10
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4

In [35]:
activities = pd.read_csv('drive/MyDrive/CASAS400/activities.csv')['activity_type'].unique()
dim_label = pd.DataFrame(columns=['label', 'dimension'])

count = 0

for a in activities:
  count = 0
  for item in dataset:
    if item.y[0] == a:
      count = count + 1
  
  dim_label = dim_label.append({'label': a, 'dimension': count}, ignore_index = True)

# Split train e test

In [36]:
def train_test_split(data, dim_label):

  train_mask = np.full(len(data), False)
  test_mask = np.full(len(data), False)

  for _, d in dim_label.iterrows():
    num_element = int(d['dimension'] * 0.8)

    for i in range(len(data)):

      if data[i].y[0] == d['label']:
        if num_element > 0:
          train_mask[i] = True
          num_element = num_element - 1
        
        else:
          test_mask[i] = True
    
  return train_mask, test_mask

In [39]:
train = []
test = []

train_mask, test_mask = train_test_split(dataset, dim_label)

for i in range(len(dataset)):
  if train_mask[i] == True:
    train.append(dataset[i])
  
  else:
    test.append(dataset[i])

In [40]:
train_loader = DataLoader(train, batch_size=64, shuffle=True)
test_loader = DataLoader(test, batch_size=64, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 64
DataBatch(x=[238, 74], edge_index=[2, 226], y=[64], batch=[238], ptr=[65])

Step 2:
Number of graphs in the current batch: 64
DataBatch(x=[255, 74], edge_index=[2, 214], y=[64], batch=[255], ptr=[65])

Step 3:
Number of graphs in the current batch: 64
DataBatch(x=[265, 74], edge_index=[2, 234], y=[64], batch=[265], ptr=[65])

Step 4:
Number of graphs in the current batch: 64
DataBatch(x=[275, 74], edge_index=[2, 263], y=[64], batch=[275], ptr=[65])

Step 5:
Number of graphs in the current batch: 64
DataBatch(x=[270, 74], edge_index=[2, 352], y=[64], batch=[270], ptr=[65])

Step 6:
Number of graphs in the current batch: 64
DataBatch(x=[263, 74], edge_index=[2, 303], y=[64], batch=[263], ptr=[65])

Step 7:
Number of graphs in the current batch: 64
DataBatch(x=[258, 74], edge_index=[2, 257], y=[64], batch=[258], ptr=[65])

Step 8:
Number of graphs in the current batch: 64
DataBatch(x=[235, 74], edge_index=[2, 258], y=[64], batch=[235], ptr

In [45]:
dim_label

,label,dimension
0,1,339
1,2,316
2,3,302
3,4,327
4,5,342
5,6,290
6,7,328
7,8,342
8,9,319
9,10,329


In [43]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(74, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 16)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): GCNConv(74, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=16, bias=True)
)


In [47]:
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data.x, data.edge_index, data.batch)  
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 171):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

<IPython.core.display.Javascript object>

ValueError: ignored